In [34]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.optim as optim

import matplotlib.pyplot as plt
%matplotlib inline

# hyperparameter 설정

In [35]:
batch_size = 16
learning_rate = 0.002
num_epoch = 200

# model 정의

In [36]:
class myCNN(nn.Module):
    def __init__(self):
        super(myCNN, self).__init__()
        self.cnn_layer = nn.Sequential(
            nn.Conv2d(1, 16, 5, padding=2),  # 28 x 28 x 16
            nn.ReLU(),

            nn.Conv2d(16, 32, 5, padding=2), # 28 x 28 x 32
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # 14 x 14 x 32

            nn.Conv2d(32, 64, 5, padding=2), # 14 x 14 x 64
            nn.ReLU(),
            nn.MaxPool2d(2, 2) # 7 x 7 x 64
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*7*7, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )

    def forward(self, x):
        out = self.cnn_layer(x)
        out = out.view(batch_size, -1) # 한줄로 펴준다
        out = self.fc_layer(out)
        return out

# data loading

In [37]:
mnist_train = dset.MNIST('', train=True, transform=transforms.ToTensor(),
                         target_transform=None, download=True)
mnist_test = dset.MNIST('', train=False, transform=transforms.ToTensor(),
                        target_transform=None, download=True)

# data loader 설정


In [38]:
train_loader = torch.utils.data.DataLoader(list(mnist_train)[:batch_size], batch_size = batch_size,
                                           shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader((mnist_test), batch_size = batch_size,
                                           shuffle=True, num_workers=2, drop_last=True)

# optimizer 설정

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = myCNN().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# accuracy 측정함수

In [40]:
def EstimateAccuracy(dloader, imodel):
    correct = 0
    total = 0

    for image, label in dloader:
        x = Variable(image, volatile=True).to(device)  # volaatile = True는 순전파만 하겠다는 뜻
        y_ = Variable(label).to(device)

        y_hat = imodel.forward(x)
        _, y_hat_index = torch.max(y_hat, 1)

        total += label.size(0)
        correct += (y_hat_index == y_).sum().float()

    print('Accuracy: {}'.format(100*correct/total))
    return 100*correct/total

# 훈련

In [41]:
loss_arr = []
acc_arr = []

for i in range(num_epoch):
    for image, label in train_loader:
        x = Variable(image).to(device)
        y_ = Variable(label).to(device)

        optimizer.zero_grad()
        y_hat = model.forward(x)
        loss = loss_func(y_hat, y_)
        loss.backward()
        optimizer.step()

        if i%10==0:
            print(i, loss)
            acc = EstimateAccuracy(test_loader, model)
            loss_arr.append(loss)
            acc_arr.append(acc)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


0 tensor(2.3177, grad_fn=<NllLossBackward0>)


<ipython-input-40-67f3e4f96341>:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  x = Variable(image, volatile=True).to(device)  # volaatile = True는 순전파만 하겠다는 뜻
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Accuracy: 11.359999656677246
10 tensor(2.3157, grad_fn=<NllLossBackward0>)
Accuracy: 11.420000076293945
20 tensor(2.3137, grad_fn=<NllLossBackward0>)
Accuracy: 11.520000457763672
30 tensor(2.3118, grad_fn=<NllLossBackward0>)
Accuracy: 11.609999656677246
40 tensor(2.3099, grad_fn=<NllLossBackward0>)
Accuracy: 11.680000305175781
50 tensor(2.3081, grad_fn=<NllLossBackward0>)


KeyboardInterrupt: 